In [1]:
import numpy as np
# 导库
import pandas as pd
from sklearn.neighbors import BallTree

In [2]:
# 读取位置信息编码好了的文件
communities = pd.read_csv('communities_with_coords.csv')
communities.head()

,小区名字,小区地址,均价,容积率,有无停车位,物业费_元/平方米/月,绿化率_percent,latitude,longitude
0,盈彩美居,天河-东圃-盈溪路,54560,2.88,1,1.20,30.0,23.124325,113.415697
1,兰亭盛荟,天河-东圃-东圃明镜路,70093,3.80,1,3.50,62.0,23.124325,113.415697
2,中海康城花园,天河-黄村-广园快速路,45544,1.74,1,2.37,40.0,23.132456,113.419568
3,宝翠园,天河-龙洞-宝翠路4号,58641,2.66,1,1.90,45.2,23.196825,113.369955
4,东方新世界,天河-天河公园-中山大道西158号,84828,3.99,1,2.30,42.0,23.139163,113.366326


In [3]:
communities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2282 entries, 0 to 2281
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   小区名字         2282 non-null   object 
 1   小区地址         2282 non-null   object 
 2   均价           2282 non-null   int64  
 3   容积率          2282 non-null   float64
 4   有无停车位        2282 non-null   int64  
 5   物业费_元/平方米/月  2282 non-null   float64
 6   绿化率_percent  2282 non-null   float64
 7   latitude     2282 non-null   float64
 8   longitude    2282 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 160.6+ KB


In [4]:
communities.describe()

,均价,容积率,有无停车位,物业费_元/平方米/月,绿化率_percent,latitude,longitude
count,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000
mean,40237.745837,2.446249,0.671779,1.274308,26.920324,23.119581,113.306912
std,15105.768891,1.424550,0.469669,1.249497,9.289624,0.024205,0.042481
min,9400.000000,0.300000,0.000000,0.200000,10.000000,23.052672,113.208664
25%,28200.000000,1.500000,0.000000,0.600000,20.000000,23.098787,113.273426
50%,37600.000000,2.180000,1.000000,0.900000,30.000000,23.127941,113.311091
75%,50000.000000,2.807500,1.000000,1.500000,30.000000,23.135260,113.333209
max,85387.000000,9.490000,1.000000,15.000000,80.000000,23.215595,113.426950


In [13]:
bus = pd.read_csv('poi/bus_stop.csv')
hospital = pd.read_csv('poi/hospital.csv')
mall = pd.read_csv('poi/mall.csv')
metro = pd.read_csv('poi/metro.csv')
primary = pd.read_csv('poi/primary_school.csv')
middle = pd.read_csv('poi/middle_school.csv')
shopping = pd.read_csv('poi/shopping.csv')
park = pd.read_csv('poi/park.csv')
cbd = pd.DataFrame({
    '地名': ['珠江新城'],
    'gcj02_y': [23.120434000000024],
    'gcj02_x': [113.32466199999999]
})
s_cbd = pd.DataFrame({
    '地名': ['琶洲', '金融城', '环市东', '白鹅潭'],
    'gcj02_y': [23.10257000000001,23.111203,23.134558999999985,23.100502872048988],
    'gcj02_x': [113.34662100000003,113.25872600000002 ,113.31183400000009,113.24032521727986]
})

In [8]:
# 检查数据特征 变量类型
bus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       1954 non-null   object 
 1   name     1954 non-null   object 
 2   gcj02_x  1954 non-null   float64
 3   gcj02_y  1954 non-null   float64
 4   adname   1954 non-null   object 
 5   中类       1954 non-null   object 
 6   小类       1954 non-null   object 
dtypes: float64(2), object(5)
memory usage: 107.0+ KB


In [16]:
# 定义求最短距离的函数
def calculate_min_distances(community, poi):
    # 将经纬度转化为弧度
    community_coords = np.radians(np.c_[community['latitude'], community['longitude']])
    poi_coords = np.radians(np.c_[poi['gcj02_y'], poi['gcj02_x']])
    # 使用BALLTree快速查询最短haversine距离
    tree = BallTree(poi_coords, metric='haversine')
    min_dist, _ = tree.query(community_coords, k=1)
    # 将距离从弧度转换为米
    min_dist_in_meters = min_dist * 6371000
    # 地球半径约为6371公里
    return min_dist_in_meters

In [17]:
# 循环求得小区到各种poi的最短距离
pois = { 'BUS': bus, 'HOS': hospital, 'MAR': mall, 'SUB': metro, 'PSC': primary, 'MSC': middle, 'PAR': park, 'CBD': cbd, 'SCBD': s_cbd }
for name, poi in pois.items():
    # 计算最短距离
    communities[f'{name}'] = calculate_min_distances(communities, poi)

In [20]:
communities.head()

,小区名字,小区地址,均价,容积率,有无停车位,物业费_元/平方米/月,绿化率_percent,latitude,longitude,BUS,HOS,MAR,SUB,PSC,MSC,PAR,CBD,SCBD
0,盈彩美居,天河-东圃-盈溪路,54560,2.88,1,1.20,30.0,23.124325,113.415697,256.509826,1535.484711,175.643977,1196.714612,417.639070,856.055562,161.120353,9319.496930,7467.040911
1,兰亭盛荟,天河-东圃-东圃明镜路,70093,3.80,1,3.50,62.0,23.124325,113.415697,256.509826,1535.484711,175.643977,1196.714612,417.639070,856.055562,161.120353,9319.496930,7467.040911
2,中海康城花园,天河-黄村-广园快速路,45544,1.74,1,2.37,40.0,23.132456,113.419568,152.513554,2100.395942,451.259944,1207.328398,388.567987,383.483784,703.540475,9796.623781,8166.680740
3,宝翠园,天河-龙洞-宝翠路4号,58641,2.66,1,1.90,45.2,23.196825,113.369955,23.832257,5297.997351,346.016106,713.996616,423.282983,382.525096,362.763426,9674.438398,9123.633151
4,东方新世界,天河-天河公园-中山大道西158号,84828,3.99,1,2.30,42.0,23.139163,113.366326,219.307358,265.723923,300.167413,1482.307992,433.038452,1132.855629,1241.665762,4742.191821,4540.610699


In [21]:
communities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2282 entries, 0 to 2281
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   小区名字         2282 non-null   object 
 1   小区地址         2282 non-null   object 
 2   均价           2282 non-null   int64  
 3   容积率          2282 non-null   float64
 4   有无停车位        2282 non-null   int64  
 5   物业费_元/平方米/月  2282 non-null   float64
 6   绿化率_percent  2282 non-null   float64
 7   latitude     2282 non-null   float64
 8   longitude    2282 non-null   float64
 9   BUS          2282 non-null   float64
 10  HOS          2282 non-null   float64
 11  MAR          2282 non-null   float64
 12  SUB          2282 non-null   float64
 13  PSC          2282 non-null   float64
 14  MSC          2282 non-null   float64
 15  PAR          2282 non-null   float64
 16  CBD          2282 non-null   float64
 17  SCBD         2282 non-null   float64
dtypes: float64(14), int64(2), object(2)
memory usage

In [22]:
communities.describe()

,均价,容积率,有无停车位,物业费_元/平方米/月,绿化率_percent,latitude,longitude,BUS,HOS,MAR,SUB,PSC,MSC,PAR,CBD,SCBD
count,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000,2282.000000
mean,40237.745837,2.446249,0.671779,1.274308,26.920324,23.119581,113.306912,179.337730,886.781860,382.182266,600.423474,343.797057,497.059662,580.858998,5014.631048,2916.806187
std,15105.768891,1.424550,0.469669,1.249497,9.289624,0.024205,0.042481,120.899622,630.204225,248.912786,335.232209,230.334008,280.694802,326.687198,2064.944810,1469.066826
min,9400.000000,0.300000,0.000000,0.200000,10.000000,23.052672,113.208664,1.430147,3.009808,7.658327,5.260690,18.551159,15.528311,29.729466,232.514434,134.606593
25%,28200.000000,1.500000,0.000000,0.600000,20.000000,23.098787,113.273426,91.921348,540.459739,216.904427,390.846092,186.120085,272.424849,318.336612,3499.263205,1910.108547
50%,37600.000000,2.180000,1.000000,0.900000,30.000000,23.127941,113.311091,148.476531,844.907028,355.943918,581.874580,259.209762,488.507183,579.382556,4617.631612,2843.000627
75%,50000.000000,2.807500,1.000000,1.500000,30.000000,23.135260,113.333209,228.074344,1067.882953,525.915025,755.667747,463.104062,728.320044,711.810631,6214.620668,3542.110515
max,85387.000000,9.490000,1.000000,15.000000,80.000000,23.215595,113.426950,1912.894039,6997.196089,2491.511105,2957.970484,2572.033278,2978.736912,3428.642535,12916.760677,10868.734624


In [23]:
communities.to_csv('communities_with_distance.csv', index=False)